Problem definition: https://adventofcode.com/2018/day/15

# Part 1

In [1426]:
class Creature:
    def __init__(self, kind, x, y, ap=3, hp=200):
        self.x = x
        self.y = y
        self.kind = kind
        self.hp = hp
        self.ap = ap
    
    def make_turn(self, grid, enemies):
        if self.hp < 0:
            return False
        if self._adjacent_enemies(grid):            
            enemy = self._find_enemy_to_atack(grid)            
            self.attack(enemy)
            return True
        else:
            x, y = self._find_next_move(grid, enemies)            
            if x and y:
                old_x, old_y = self.x, self.y
                self.move(x, y)
                grid[y][x] = self
                grid[old_y][old_x] = '.'
                if self._adjacent_enemies(grid):            
                    enemy = self._find_enemy_to_atack(grid)            
                    self.attack(enemy)
                return True
        return False

    def move(self, x, y):
        self.x, self.y = x, y
    
    def attack(self, enemy):
        enemy.hp -= self.ap
    
    def _adjacent_enemies(self, grid): 
        x, y = self.x, self.y
        enemies = []
        enemy_kind = 'E' if self.kind == 'G' else 'G'
        if x > 0 and type(grid[y][x - 1]) == Creature and grid[y][x - 1].kind == enemy_kind:
            enemies.append(grid[y][x - 1])
        if x < len(grid[0]) - 1 and type(grid[y][x + 1]) == Creature and grid[y][x + 1].kind == enemy_kind:
            enemies.append(grid[y][x + 1])
        if y > 0 and type(grid[y - 1][x]) == Creature and grid[y - 1][x].kind == enemy_kind:
            enemies.append(grid[y - 1][x])
        if y < len(grid) - 1 and type(grid[y + 1][x]) == Creature and grid[y + 1][x].kind == enemy_kind:
            enemies.append(grid[y + 1][x])
        return enemies
    
    
    def _find_next_move(self, grid, enemies):
        target_coords = []
        for enemy in enemies:            
            target_coords.extend(get_succ(grid, enemy.x, enemy.y))
        target_coords = set(target_coords)
        graph = [row[:] for row in grid]
        succ = get_succ(graph, self.x, self.y)
        distance = {(self.x, self.y): (0, None)}
        step = 1
        for s in succ:
            graph[s[1]][s[0]] = step
            distance[s] = (step, (self.x, self.y))
            
        queue = succ
        while queue:
            coord = queue.pop(0)
            step = graph[coord[1]][coord[0]] + 1
            succ = get_succ(graph, coord[0], coord[1])
            for s in succ:
                graph[s[1]][s[0]] = step
                distance[s] = (step, (coord[0], coord[1]))

            queue.extend(succ)

        try:
            min_dist, closest_y, closest_x = min((dist, pos[1], pos[0]) for pos, (dist, parent) in distance.items() if pos in target_coords)
        except ValueError:
            return None, None
        closest = (closest_x, closest_y)
        while distance[closest][0] > 1:
            closest = distance[closest][1]
        return closest
        
    def _find_enemy_to_atack(self, grid):
        enemies = sorted(self._adjacent_enemies(grid), key=lambda c: (c.hp, c.y, c.x))
        return enemies[0]
        
    def __str__(self):
        return self.kind
    
    def __repr__(self):
        return '{{{}: pos:({}, {}), hp: {}}}'.format(self.kind, self.x, self.y, self.hp)

Graph utils

In [1427]:
def get_succ(graph, x, y):
    succ = []
    if y > 0 and graph[y - 1][x] == '.':
        succ.append((x, y - 1))
    if x > 0 and graph[y][x - 1] == '.':
        succ.append((x - 1, y))
    if x < len(graph[0]) - 1 and graph[y][x + 1] == '.':
        succ.append((x + 1, y))
    if y < len(graph) - 1 and graph[y + 1][x] == '.':
        succ.append((x, y + 1)) 
    return succ

In [1428]:
def build_grid(filename, elf_ap=3):
    elves = []
    goblins = []
    grid = []
    f = open(filename, 'r')
    x, y = 0, 0
    for y, line in enumerate(f.readlines()):
        grid.append([])
        for x, c in enumerate(line.strip('\n')):            
            if c == 'G':
                cell = Creature(c, x, y)
                goblins.append(cell)
            elif c == 'E':
                cell = Creature(c, x, y, elf_ap)
                elves.append(cell)
            else:              
                cell = c
            grid[y].append(cell)
    return grid, elves, goblins

In [1429]:
def print_grid(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            print(grid[i][j], end='')
        print('')

In [1430]:
grid, elves, goblins = build_grid('example1.txt')
print_grid(grid)

#######
#E..G.#
#...#.#
#.G.#G#
#######


In [1431]:
elf = elves[0]

In [1432]:
elf.make_turn(grid, goblins)
print_grid(grid)

#######
#.E.G.#
#...#.#
#.G.#G#
#######


In [1433]:
elf.make_turn(grid, goblins)
print_grid(grid)

#######
#..EG.#
#...#.#
#.G.#G#
#######


In [1434]:
elf.make_turn(grid, goblins)
print_grid(grid)

#######
#..EG.#
#...#.#
#.G.#G#
#######


In [1436]:
grid, elves, goblins = build_grid('example2.txt')
print_grid(grid)

#######
#.E...#
#.....#
#...G.#
#######


In [1437]:
elf = elves[0]
elf.make_turn(grid, goblins)
print_grid(grid)

#######
#..E..#
#.....#
#...G.#
#######


In [1438]:
elf.make_turn(grid, goblins)
print_grid(grid)

#######
#...E.#
#.....#
#...G.#
#######


In [1439]:
elf.make_turn(grid, goblins)
print_grid(grid)

#######
#.....#
#...E.#
#...G.#
#######


In [1440]:
def main_loop(grid, elves, goblins, rounds = 0, should_print=False):
    curr_round = 0
    while elves and goblins:
        queue = get_queue(elves, goblins)
        while queue:
            creature = queue.pop(0)
            enemies = elves if creature.kind == 'G' else goblins
            creature.make_turn(grid, enemies)
            
            for elf in elves:
                if elf.hp <= 0:
                    grid[elf.y][elf.x] = '.'
            for goblin in goblins:
                if goblin.hp <= 0:
                    grid[goblin.y][goblin.x] = '.'

            elves = [elf for elf in elves if elf.hp > 0]
            goblins = [goblin for goblin in goblins if goblin.hp > 0]
            if queue and (len(elves) == 0 or len(goblins) == 0):
                return curr_round
            
        if should_print:
            print_grid(grid)        
        
        curr_round += 1
        if rounds > 0 and curr_round == rounds:
            break
    return curr_round

In [1441]:
def get_queue(elves, goblins):
    return sorted(elves + goblins, key=lambda c: (c.y, c.x))

In [1442]:
grid, elves, goblins = build_grid('example3.txt')
print_grid(grid)

#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########


In [1443]:
main_loop(grid, elves, goblins, rounds=3, should_print=True)

#########
#.G...G.#
#...G...#
#...E..G#
#.G.....#
#.......#
#G..G..G#
#.......#
#########
#########
#..G.G..#
#...G...#
#.G.E.G.#
#.......#
#G..G..G#
#.......#
#.......#
#########
#########
#.......#
#..GGG..#
#..GEG..#
#G..G...#
#......G#
#.......#
#.......#
#########


3

In [1444]:
grid, elves, goblins = build_grid('example3.txt')
print_grid(grid)

#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########


In [1445]:
total_rounds = main_loop(grid, elves, goblins)

In [1446]:
print_grid(grid)

#########
#.......#
#..GGG..#
#..G.G..#
#G..G...#
#......G#
#.......#
#.......#
#########


In [1447]:
elves

[{E: pos:(4, 3), hp: -1}]

In [1448]:
grid, elves, goblins = build_grid('example4.txt')
print_grid(grid)

#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######


In [1449]:
total_rounds = main_loop(grid, elves, goblins)

In [1450]:
total_rounds

47

In [1451]:
total_rounds = main_loop(grid, elves, goblins, rounds=23, should_print=False)
print(elves)
print(goblins)

[{E: pos:(4, 2), hp: -1}, {E: pos:(5, 4), hp: -1}]
[{G: pos:(2, 1), hp: 200}, {G: pos:(2, 2), hp: 131}, {G: pos:(5, 3), hp: 59}, {G: pos:(5, 5), hp: 200}]


In [1452]:
grid, elves, goblins = build_grid('example4.txt')
total_rounds = main_loop(grid, elves, goblins, rounds=24, should_print=False)
print(elves)
print(goblins)

[{E: pos:(4, 2), hp: -1}, {E: pos:(5, 4), hp: 128}]
[{G: pos:(3, 1), hp: 200}, {G: pos:(4, 2), hp: 131}, {G: pos:(5, 3), hp: 128}, {G: pos:(3, 3), hp: 200}]


In [1453]:
# round 25
total_rounds = main_loop(grid, elves, goblins, rounds=1, should_print=False)
print(elves)
print(goblins)

[{E: pos:(4, 2), hp: -1}, {E: pos:(5, 4), hp: 125}]
[{G: pos:(4, 1), hp: 200}, {G: pos:(3, 2), hp: 131}, {G: pos:(5, 3), hp: 125}, {G: pos:(3, 4), hp: 200}]


In [1454]:
# round 26
total_rounds = main_loop(grid, elves, goblins, rounds=1, should_print=False)
print(elves)
print(goblins)

[{E: pos:(4, 2), hp: -1}, {E: pos:(5, 4), hp: 122}]
[{G: pos:(5, 1), hp: 200}, {G: pos:(2, 2), hp: 131}, {G: pos:(5, 3), hp: 122}, {G: pos:(3, 5), hp: 200}]


In [1455]:
print_grid(grid)

#######
#....G#
#.G...#
#.#.#G#
#...#E#
#..G..#
#######


In [1456]:
grid, elves, goblins = build_grid('example4.txt')
print_grid(grid)
total_rounds = main_loop(grid, elves, goblins)

#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######


In [1457]:
print_grid(grid)

#######
#G....#
#.G...#
#.#.#G#
#...#.#
#....G#
#######


In [1458]:
total_rounds

47

In [1459]:
goblins

[{G: pos:(1, 1), hp: 200},
 {G: pos:(2, 2), hp: 131},
 {G: pos:(5, 3), hp: 59},
 {G: pos:(5, 5), hp: 200}]

In [1460]:
sum([g.hp for g in goblins if g.hp > 0]) * (total_rounds)

27730

In [1461]:
grid, elves, goblins = build_grid('example5.txt')
print_grid(grid)
total_rounds = main_loop(grid, elves, goblins)

#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######


In [1462]:
print_grid(grid)

#######
#...#E#
#E#...#
#.E##.#
#E..#E#
#.....#
#######


In [1463]:
total_rounds

37

In [1464]:
sum([g.hp for g in elves if g.hp > 0] + []) * (total_rounds)

36334

In [1465]:
elves

[{E: pos:(5, 1), hp: 200},
 {E: pos:(1, 2), hp: -1},
 {E: pos:(1, 2), hp: 197},
 {E: pos:(5, 4), hp: 200},
 {E: pos:(1, 4), hp: 200},
 {E: pos:(2, 3), hp: 185}]

In [1466]:
grid, elves, goblins = build_grid('example5.txt')
print_grid(grid)
total_rounds = main_loop(grid, elves, goblins, rounds=37, should_print=True)

#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######
#######
#G.E#E#
#E#..E#
#G.##.#
#...#E#
#..E..#
#######
#######
#GE.#E#
#E#..E#
#G.##.#
#..E#E#
#.....#
#######
#######
#GE.#E#
#E#..E#
#G.##.#
#.E.#.#
#....E#
#######
#######
#GE.#E#
#E#..E#
#GE##.#
#...#.#
#...E.#
#######
#######
#GE.#E#
#E#..E#
#GE##.#
#...#.#
#..E..#
#######
#######
#GE.#E#
#E#..E#
#GE##.#
#..E#.#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##E#
#.E.#.#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#
#######
#######
#GE.#E#
#E#...#
#GE##.#
#E..#E#
#.....#


In [1467]:
goblins

[{G: pos:(1, 1), hp: -1}, {G: pos:(1, 3), hp: 5}]

In [1468]:
sum([200, 197, 200, 200, 185]) * 37

36334

In [1469]:
grid, elves, goblins = build_grid('example6.txt')
print_grid(grid)
total_rounds = main_loop(grid, elves, goblins)

#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######


In [1470]:
total_rounds

46

In [1471]:
sum([c.hp for c in elves if c.hp > 0]) * total_rounds

39514

In [1472]:
grid, elves, goblins = build_grid('input.txt')
print_grid(grid)
total_rounds = main_loop(grid, elves, goblins)

################################
#########....G#######.##########
##########.G########...#########
###########.########.#.#########
###########.#..G######..######.#
##########..#...###......G..##.#
##.#######......#..G....E#.....#
##.##..######...........E..E####
##.##...###................#####
#.....G.G...........G.....######
#...G....G...................###
#G.G.............EG..........###
#..G..........#####.........####
##.G.......G.#######.........###
#####....G..#########..G.E....##
####....#...#########.........##
#######.##..#########...#.....##
#########...#########.....######
##########..#########G....######
##########...#######..E...######
##########....#####...#EE.######
#########.........G.......######
#########............###########
############..........##########
############.......#.###########
###########...........##########
###########........#.###########
##########E.#......#############
#########...##....E.############
#########.######....############
##########

In [1473]:
total_rounds

79

In [1474]:
sum([g.hp for g in goblins if g.hp > 0]) * total_rounds

207059

In [1475]:
grid, elves, goblins = build_grid('example7.txt')
print_grid(grid)
total_rounds = main_loop(grid, elves, goblins)

#########
#G......#
#.E.#...#
#..##..G#
#...##..#
#...#...#
#.G...G.#
#.....G.#
#########


In [1476]:
total_rounds

20

# Part 2

In [1423]:
def find_ap_withouth_single_death(filename):
    elves_won = False
    ap = 4
    grid, elves, goblins = build_grid(filename, ap)
    while not elves_won:        
        try:
            total_rounds = main_loop2(grid, elves, goblins)
            elves_won = True
        except ElfDied:        
            ap += 1
            grid, elves, goblins = build_grid(filename, ap)
    print_grid(grid)
    print('ap: ', ap)
    print('total rounds: ', total_rounds)
    print('elves: ', elves)
    return total_rounds * sum([e.hp for e in elves])
    
def main_loop2(grid, elves, goblins, rounds = 0, should_print=False):
    curr_round = 0
    while elves and goblins:
        queue = get_queue(elves, goblins)
        while queue:
            creature = queue.pop(0)
            enemies = elves if creature.kind == 'G' else goblins
            creature.make_turn(grid, enemies)
            
            for elf in elves:
                if elf.hp <= 0:
                    raise ElfDied
            for goblin in goblins:
                if goblin.hp <= 0:
                    grid[goblin.y][goblin.x] = '.'

            elves = [elf for elf in elves if elf.hp > 0]
            goblins = [goblin for goblin in goblins if goblin.hp > 0]
            if queue and (len(elves) == 0 or len(goblins) == 0):
                return curr_round
            
        if should_print:
            print_grid(grid)        
        
        curr_round += 1
        if rounds > 0 and curr_round == rounds:
            break
    return curr_round

class ElfDied(Exception):
    pass

In [1477]:
find_ap_withouth_single_death('example8.txt')

#######
#..E..#
#...E.#
#.#.#.#
#...#.#
#.....#
#######
ap:  15
total rounds:  29
elves:  [{E: pos:(4, 2), hp: 14}, {E: pos:(3, 1), hp: 158}]


4988

In [1478]:
find_ap_withouth_single_death('example9.txt')

#######
#.E.E.#
#.#E..#
#E.##E#
#.E.#.#
#...#.#
#######
ap:  4
total rounds:  33
elves:  [{E: pos:(2, 1), hp: 200}, {E: pos:(4, 1), hp: 23}, {E: pos:(3, 2), hp: 200}, {E: pos:(1, 3), hp: 122}, {E: pos:(5, 3), hp: 200}, {E: pos:(2, 4), hp: 200}]


31185

In [1479]:
find_ap_withouth_single_death('example10.txt')

#######
#.E.#.#
#.#E..#
#..#..#
#...#.#
#.....#
#######
ap:  15
total rounds:  37
elves:  [{E: pos:(2, 1), hp: 8}, {E: pos:(3, 2), hp: 86}]


3478

In [1425]:
find_ap_withouth_single_death('input.txt')

################################
#########.....#######.##########
##########..########...#########
###########.########.#.#########
###########.#...######..######.#
##########..#...###.........##.#
##.#######......#........#.....#
##.##..######....E..........####
##.##...###.....E.EE.......#####
#..............E.E........######
#...............E............###
#.........EE........E........###
#.............#####.........####
##...........#######.........###
#####.......#########.........##
####....#...#########.........##
#######.##..#########...#.....##
#########...#########.....######
##########..#########.....######
##########...#######......######
##########....#####...#...######
#########.................######
#########............###########
############..........##########
############.......#.###########
###########...........##########
###########........#.###########
##########..#......#############
#########...##......############
#########.######....############
##########

49120